## Load packages and modules

In [27]:
import numpy as np
import imageio.v2 as imageio
from osgeo import gdal
import os
from glob import glob
from skimage import measure
import pandas as pd
import subprocess
import sys
from shapely.geometry import box
import rioxarray
import datacube
module_paths=['../1_Identify_months_thresholds_model_evaluation']
for module_path in module_paths:
    if module_path not in sys.path:
        sys.path.append(module_path)

## Define parameters

In [3]:
# import functions from modules
from datasets import export_geotiff
from instance_segment import InstSegm

In [23]:
tiles_shp='Rwanda_tiles_edited.shp'
extent_mosaic='results/processed/Rwanda_extent_prob_2021_04_08_12_mosaic.tif'
bound_mosaic='results/processed/Rwanda_bound_prob_2021_04_08_12_mosaic.tif'
grd_search_df='../1_Identify_months_thresholds_model_evaluation/results/averaged/Rwanda_grid_search_thresholds.csv' # grid search results of thresholds
country = 'Rwanda'
str_year='2021'

In [5]:
out_folder='results_retiled'
if not os.path.isdir(out_folder):
    os.makedirs(out_folder)

## Read in tiles and grid searched thresholds

In [20]:
tiles=gpd.read_file(tiles_shp)
bboxes=tiles.bounds
crs=tiles.crs.to_string()
tiles

,id,left,top,right,bottom,geometry
0,2.0,706963.175735,9.834132e+06,756963.175735,9.784132e+06,"POLYGON ((735394.049 9825116.169, 757031.306 9..."
1,3.0,706963.175735,9.784152e+06,756963.175735,9.734152e+06,"POLYGON ((706963.176 9784152.365, 756963.176 9..."
2,4.0,706963.175735,9.734172e+06,756963.175735,9.684172e+06,"POLYGON ((706963.176 9734172.365, 756963.176 9..."
3,5.0,756943.175735,9.884112e+06,806943.175735,9.834112e+06,"POLYGON ((756926.514 9851826.590, 806961.578 9..."
4,6.0,756943.175735,9.834132e+06,806943.175735,9.784132e+06,"POLYGON ((756943.176 9834132.365, 806943.176 9..."
5,7.0,756943.175735,9.784152e+06,806943.175735,9.734152e+06,"POLYGON ((756943.176 9784152.365, 806943.176 9..."
6,8.0,756943.175735,9.734172e+06,806943.175735,9.684172e+06,"POLYGON ((756943.176 9734172.365, 806943.176 9..."
7,9.0,806923.175735,9.884112e+06,856923.175735,9.834112e+06,"POLYGON ((806856.531 9860361.702, 856926.245 9..."
8,10.0,806923.175735,9.834132e+06,856923.175735,9.784132e+06,"POLYGON ((806923.176 9834132.365, 856923.176 9..."
9,11.0,806923.175735,9.784152e+06,856923.175735,9.734152e+06,"POLYGON ((806923.176 9784152.365, 856923.176 9..."


In [24]:
# read in or provide the best thresholds
if not grd_search_df is None:
    hp_df=pd.read_csv(grd_search_df)
    t_ext_best=hp_df.iloc[hp_df['mIoU'].idxmax()]['t_ext']
    t_bnd_best=hp_df.iloc[hp_df['mIoU'].idxmax()]['t_bound']
else:
    t_ext_best=0.3
    t_bnd_best=0.1

In [36]:
n_features

1228

In [38]:
instances_labelled.max()

1229

## Retile mosaic of predictions, do segmentation and masking

In [ ]:
# for index,tile in tiles.iterrows():
#     print('processing tile ',index)
#     # get bbox
#     minx,miny,maxx,maxy=bboxes.iloc[index]
#     # clip predictions mosaic using tile
#     out_bound=os.path.join(out_folder, country+'_bound_prob_tile_'+str(index)+'.tif')
#     gdal_cmd=["gdal_translate", "-of", "GTiff",
#               "-projwin",str(minx),str(maxy),str(maxx),str(miny),
#               '-projwin_srs',crs,extent_mosaic,out_bound]
#     subprocess.run(gdal_cmd,stdout=subprocess.DEVNULL)
    
#     out_extent=out_bound.replace('bound','extent')
#     gdal_cmd=["gdal_translate", "-of", "GTiff",
#           "-projwin",str(minx),str(maxy),str(maxx),str(miny),
#           '-projwin_srs',crs,extent_mosaic,out_extent]
#     subprocess.run(gdal_cmd,stdout=subprocess.DEVNULL)
    
#     # read in clipped predictions
#     ds_extent = gdal.Open(out_extent)
#     geotrans=ds_extent.GetGeoTransform()
#     proj=ds_extent.GetProjection()
#     ds_extent=None

#     extent_prob=imageio.imread(out_extent)
#     bound_prob=imageio.imread(out_bound)

#     # do segmentation
#     instances_predicted=InstSegm(extent_prob, bound_prob, t_ext=t_ext_best, t_bound=t_bnd_best)

#     # label connected regions, non-field (-1) will be labelled as 0
#     instances_labelled,n_features= measure.label(instances_predicted, background=-1,return_num=True)
#     instances_labelled+=1 # background value from -1 to 0
    
#     # export labelled instances as geotiff
#     outname=os.path.join(out_folder,os.path.basename(extent_mosaic).replace('mosaic','tile_')[:-4]+str(index)+'.tif')
#     outname=outname.replace('extent_prob','field_instance')
#     export_geotiff(outname,instances_labelled,geotrans,proj,gdal.GDT_Int32)
    
#     # mask using DE Africa crop mask
#     # load DE Africa crop mask 2019
#     xr_ds=rioxarray.open_rasterio(outname).to_dataset(name='field_instance')
#     dc = datacube.Datacube(app='cropland_extent')
#     cm = dc.load(product='crop_mask',like=xr_ds,time=('2019')).squeeze()
#     np_crop_mask=cm['mask'].to_numpy()
#     # reassign noncrop instances as background (0)
#     n_instances=np.max(instances_labelled)
#     instances_labelled_masked=instances_labelled.copy()
#     for value in range(1,n_instances+1):
#         if np.nansum((instances_labelled==value)&(np_crop_mask==1))==0:
#             instances_labelled_masked[instances_labelled==value]=0
    
#     # export masked as geotiff
#     outname=outname[:-4]+'_masked.tif'
#     export_geotiff(outname,instances_labelled_masked,geotrans,proj,gdal.GDT_Int32)

In [34]:
for index,tile in tiles.iterrows():
    print('processing tile ',index)
    # get bbox
    minx,miny,maxx,maxy=bboxes.iloc[index]
    # clip predictions mosaic using tile
    out_bound=os.path.join(out_folder, country+'_bound_prob_tile_'+str(index)+'.tif')
    gdal_cmd=["gdal_translate", "-of", "GTiff",
              "-projwin",str(minx),str(maxy),str(maxx),str(miny),
              '-projwin_srs',crs,extent_mosaic,out_bound]
    subprocess.run(gdal_cmd,stdout=subprocess.DEVNULL)
    
    out_extent=out_bound.replace('bound','extent')
    gdal_cmd=["gdal_translate", "-of", "GTiff",
          "-projwin",str(minx),str(maxy),str(maxx),str(miny),
          '-projwin_srs',crs,extent_mosaic,out_extent]
    subprocess.run(gdal_cmd,stdout=subprocess.DEVNULL)
    
    # read in clipped predictions
    ds_extent = gdal.Open(out_extent)
    geotrans=ds_extent.GetGeoTransform()
    proj=ds_extent.GetProjection()
    ds_extent=None

    extent_prob=imageio.imread(out_extent)
    bound_prob=imageio.imread(out_bound)

    # do segmentation
    instances_predicted=InstSegm(extent_prob, bound_prob, t_ext=t_ext_best, t_bound=t_bnd_best)

    # label connected regions, non-field (-1) will be labelled as 0
    instances_labelled,n_features= measure.label(instances_predicted, background=-1,return_num=True)
    instances_labelled+=1 # background value from -1 to 0
    
    # export labelled instances as geotiff
    outname=os.path.join(out_folder,os.path.basename(extent_mosaic).replace('mosaic','tile_')[:-4]+str(index)+'.tif')
    outname=outname.replace('extent_prob','field_instance')
    export_geotiff(outname,instances_labelled,geotrans,proj,gdal.GDT_Int32)
    
    # mask using DE Africa crop mask
    # load DE Africa crop mask 2019
    xr_ds=rioxarray.open_rasterio(outname).to_dataset(name='field_instance')
    dc = datacube.Datacube(app='cropland_extent')
    cm = dc.load(product='crop_mask',like=xr_ds,time=('2019')).squeeze()
    np_crop_mask=cm['mask'].to_numpy()
    # reassign noncrop instances as background (0)
    n_instances=np.max(instances_labelled)
    instances_labelled_masked=instances_labelled.copy()
    for value in range(1,n_instances+1):
        if np.nansum((instances_labelled==value)&(np_crop_mask==1))==0:
            instances_labelled_masked[instances_labelled==value]=0
    
    # export masked as geotiff
    outname=outname[:-4]+'_masked.tif'
    export_geotiff(outname,instances_labelled_masked,geotrans,proj,gdal.GDT_Int32)

557

## Mosaic all chunks

In [ ]:
# mosaic unmasked instance results
! gdal_merge.py -o results/processed/Rwanda_field_instance_2021_04_08_12_mosaic.tif -co COMPRESS=Deflate results/processed/Rwanda*average_field_instance*_cropped_masked.tif

In [ ]:
# mosaic masked instance results
! gdal_merge.py -o results/processed/Rwanda_field_instance_2021_04_08_12_mosaic.tif -co COMPRESS=Deflate results/processed/Rwanda*average_field_instance*_cropped_masked.tif